ALS + GB
Подобрать другие retrivials

In [1]:
%autosave 60
%reload_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '..')  # noqa: E702

Autosaving every 60 seconds


In [25]:
import polars as pl
import numpy as np
import implicit
from scipy.sparse import csr_matrix
from pathlib import Path
from catboost import CatBoostClassifier
from datetime import timedelta

from tools.experiment_tracker import ExperimentTracker

In [20]:
DATA_DIR = Path.cwd().parent / "data" / "avito_ml_cup"
EVAL_DAYS_TRESHOLD = 7

In [21]:
df_test_users = pl.read_parquet(f'{DATA_DIR}/test_users.pq', n_rows=100_000)
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq', n_rows=100_000)

df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features.pq', n_rows=100_000)
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq', n_rows=100_000)
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

In [22]:
df_clickstream

cookie,item,event,event_date,platform,surface,node
i64,i64,i64,datetime[ns],i64,i64,u32
0,19915558,17,2025-02-05 02:30:59,3,2,115659
0,2680232,17,2025-01-24 21:16:57,3,2,115829
1,4247649,17,2025-01-29 23:00:58,2,2,7
1,4247649,17,2025-02-17 14:55:17,2,2,7
1,2171135,17,2025-01-17 19:23:29,2,2,214458
…,…,…,…,…,…,…
10725,8424390,17,2025-02-05 04:20:58,2,11,81455
10725,8424390,17,2025-01-31 09:30:52,2,11,81455
10725,24159200,17,2025-02-04 12:49:22,2,2,326790


In [23]:
treshhold = df_clickstream['event_date'].max() - timedelta(days=EVAL_DAYS_TRESHOLD)

df_train = df_clickstream.filter(df_clickstream['event_date']<= treshhold)

df_eval = df_clickstream.filter(df_clickstream['event_date']> treshhold)[['cookie', 'node', 'event']]
df_eval = df_eval.join(df_train, on=['cookie', 'node'], how='anti') # Делаем так, чтобы не предсказывать группы товаров, с которыми уже были взаимодействия

df_eval = df_eval.filter(
    pl.col('event').is_in(
        df_event.filter(pl.col('is_contact')==1)['event'].unique()
    )
)

df_eval = df_eval.filter(
        pl.col('cookie').is_in(df_train['cookie'].unique())
    ).filter(
        pl.col('node').is_in(df_train['node'].unique())
    )

/tmp/ipykernel_203383/1528941487.py:8: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df_eval = df_eval.filter(
/tmp/ipykernel_203383/1528941487.py:14: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df_eval = df_eval.filter(
/tmp/ipykernel_203383/1528941487.py:16: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  ).filter(


In [24]:
df_eval = df_eval.unique(['cookie', 'node'])

# Train model

## Train ALS

In [26]:
def get_als_pred(users, nodes, user_to_pred):
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()
        
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v:k for k,v in item_id_to_index.items()}
    
    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()
    
    values = [1] * len(users)
    
    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    model = implicit.als.AlternatingLeastSquares(iterations=10, factors=60)
    model.fit(sparse_matrix, )
    
    
    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])
    
    recommendations, scores = model.recommend(user4pred, sparse_matrix[user4pred], N=40, filter_already_liked_items=True)
    
    df_pred = pl.DataFrame(
        {
            'node': [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()
            ], 
             'cookie': list(user_to_pred),
            'scores': scores.tolist()
            
        }
    )
    df_pred = df_pred.explode(['node', 'scores'])
    return df_pred

In [28]:
users = df_train["cookie"]
nodes = df_train["node"]
eval_users = df_eval['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes, eval_users)

/home/silverservicepipe/projects/avito_cup_ml_2025/.venv/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 10/10 [00:01<00:00,  8.66it/s]


In [29]:
df_pred

node,cookie,scores
i64,i64,f64
229346,55,0.071387
229315,55,0.035229
229448,55,0.020347
128928,55,0.018957
229314,55,0.018756
…,…,…
262560,10721,0.015918
214377,10721,0.015828
152008,10721,0.015698


## Train catboost

### Метрика

In [30]:
def recall_at(df_true, df_pred, k=40):
    return  df_true[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']], 
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()


In [31]:
recall_at(df_eval, df_pred, k=40)

0.18435754189944134